# Channel to Channel skew control in time
This script checks the the channel-to-channel synchronization on fixed sample clock on differnt points
offsets from 0 to 1024 in fixed jumps, by reading from a Keysight MSO9254A Oscilloscope.
The threshold is 20ps for the means and 15ps for the jitter.

How to test:
connect SMA to rma cable from channel 1 of the proteus to channel 1 of the scope. Do the same thing with channel 2.(It is very important to make skew caibration to each cable before the test) run the test and wait for results.
Modifications:
it possible to do the test to every channel/marker on the proteus but need to modify the script as well.the fixied time jump  can be change also

In [1]:
## Import required Libraries
import os
import sys
import tempfile
import webbrowser
srcpath = os.path.realpath('..//..//SourceFiles')
sys.path.append(srcpath)
from teproteus import TEProteusAdmin as TepAdmin
from teproteus import TEProteusInst as TepInst
from teproteus_functions_v3 import connect
from teproteus_functions_v3 import disconnect
from teproteus_functions_v3 import set_lib_dir_path
from teproteus_functions_v3 import get_cpatured_header
from teproteus_functions_v3 import gauss_env
from teproteus_functions_v3 import iq_kernel
from teproteus_functions_v3 import pack_kernel_data
from teproteus import TEProteusAdmin, TEProteusInst
from tevisainst import TEVisaInst
# matplotlib notebook
import numpy as np
import time
import ipywidgets as widgets
from IPython.core.debugger import set_trace
from scipy.signal import chirp, sweep_poly
import matplotlib.pyplot as plt

plt.style.use('ggplot')
from scipy import signal
import math
import pdb

import pyvisa as visa
from pyvisa.errors import Error

In [10]:
# Connect to instrument
inst = connect('192.90.70.22')

# Get the instrument's *IDN
resp = inst.send_scpi_query('*IDN?')
print(resp)

# Get the model:
model_name = inst.send_scpi_query('SYST:INF:MODel?')
print('Model: {0} '.format(model_name))
# Get number of channels
resp = inst.send_scpi_query(":INST:CHAN? MAX")
#print("Number of channels: " + resp)
num_channels = int(resp)

# Get the DAC mode (8 bits or 16 bits)
resp = inst.send_scpi_query(':SYST:INF:DAC?')
if resp == 'M0':
    dac_mode = 16
else:
    dac_mode = 8

#print('DAC {0} bits'.format(dac_mode))

if dac_mode == 16:
    max_dac = 65535
    data_type = np.uint16 
else:
    max_dac = 255
    data_type = np.uint8 
    
half_dac = max_dac / 2.0

# Several initializations ..
inst.default_paranoia_level = 2
inst.send_scpi_cmd('*CLS; *RST')
inst.send_scpi_cmd(':TRACe:DELete:ALL')
# Get the maximal number of segments
resp = inst.send_scpi_query(":TRACe:SELect:SEGMent? MAX")
print("Max segment number: " + resp)
max_seg_number = int(resp)

# Get the available memory in bytes of wavform-data (per DDR):
resp = inst.send_scpi_query(":TRACe:FREE?")
arbmem_capacity = int(resp)
print("Available memory per DDR: {0:,} wave-bytes".format(arbmem_capacity))
resp = inst.send_scpi_query(":FREQ?")
print("Sampling Rate: " + resp)

Service connect
Trying to connect to IP:192.90.70.22
Tabor Electronics,P9484M,000002232771,1.238.6 --Tabor Electronics,P9484M,000000000000,1.238.6 --slot#: 2, slot#: 4, 
Model: P9484M 
Max segment number: 65536
Available memory per DDR: 17,171,480,576 wave-bytes
Sampling Rate: 1000000000.000


In [7]:
# Build waveforms
seglen = 4096
ncycles = 2
max_dac = 65535
waves = [None for _ in range(num_channels)]
# Create Square wave

for i in range(num_channels):
    cyclelen = seglen / ncycles

    x = np.linspace(start=0, stop=seglen, num=seglen, endpoint=False)
    y = np.fmod(x, cyclelen)
    y = (y <= cyclelen / 2) * max_dac
    y = np.round(y)
    y = np.clip(y, 0, max_dac)
    if dac_mode == 16:
        waves[i] = y.astype(np.uint16)
    else:
        waves[i] = y.astype(np.uint8)

# Download segments
for ii in range(num_channels):
    channb = ii + 1
    segnum = ii + 1
    wav = waves[0]
    print('Download wave to segment {0} of channel {1}'.format(segnum, channb))

    # Select channel
    inst.send_scpi_cmd(':INST:CHAN {}'.format(channb))

    # Define segment
    inst.send_scpi_cmd(':TRAC:DEF {},{}'.format(segnum, seglen))
    inst.send_scpi_cmd(':VOLT 0.5')

    # Select the segment
    cmd = ':TRAC:SEL {0}'.format(segnum)
    inst.send_scpi_cmd(cmd)

    # Send the binary-data:
    inst.write_binary_data(':TRAC:DATA', wav)

    resp = inst.send_scpi_query(':SYST:ERR?')
    resp = resp.rstrip()
    if not resp.startswith('0'):
        print('ERROR: "{0}" after writing binary values'.format(resp))

    # Play the specified segment at the selected channel:
    cmd = ':SOUR:FUNC:MODE:SEGM {0}'.format(segnum)
    inst.send_scpi_cmd(cmd)

    # Turn on the output of the selected channel:
    inst.send_scpi_cmd(':OUTP ON')

    resp = inst.send_scpi_query(':SYST:ERR?')
    print(resp)

print()


Download wave to segment 1 of channel 1
0, no error
Download wave to segment 2 of channel 2
0, no error
Download wave to segment 3 of channel 3
0, no error
Download wave to segment 4 of channel 4
0, no error



In [8]:
scope_addr2= 'USB0::0x2A8D::0x900E::MY55490134::INSTR'
# connect to scope via USB
try:
    resourceManager = visa.ResourceManager()   # Create a connection (session) to the instrument
    scope = resourceManager.open_resource(scope_addr2)
except visa.Error as ex2:
        print('Couldn\'t connect to \'%s\', exiting now...' % scope_addr2)
        sys.exit()
## scope acquisition 
# Send *IDN? and read the response
scope.write('*RST?')
scope.write('*IDN?')
idn = scope.read()
print('*IDN? returned: %s' % idn.rstrip('\n'))
scope.write("AUTOscale")

*IDN? returned: KEYSIGHT TECHNOLOGIES,MSO9254A,MY55490134,06.40.01101


11

# Channels on the same DAC (Connect the channels to be compared to the scope channel 1 and channel 2)

In [ ]:
ScopeChannel1,ScopeChannel2 =1,2 
inst.send_scpi_cmd(':INST:CHAN {0};:FREQ:RAST 2.5e9;:INIT:CONT ON;:SOUR:FUNC:MODE:SEGM 1;:OUTP ON'.format(ScopeChannel1))
inst.send_scpi_cmd(':INST:CHAN {0};:INIT:CONT ON;:SOUR:FUNC:MODE:SEGM 1;:OUTP ON'.format(ScopeChannel1))

test_success = True
scope.write('*RST;:CHAN{0}:DISP ON;:TIM:RANG 5E-9;:CHAN{1}:SCAL 0.2;:CHAN{1}:INP DC50'.format(ScopeChannel2,ScopeChannel1))
scope.write(':MEASure:STATistics ON')
scope.write(':MEASure:SENDvalid ON')
scope.write('CHAN{0}:SCAL 0.2;:CHAN{0}:INP DC50'.format(ScopeChannel2))
scope.write('FUNC1:AVER CHAN2;:FUNC1:DISP ON')
time.sleep(1)
scope.write(
    ':SYSTem:HEADer OFF')  # When receiving numeric data into numeric variables, turn off the headers. Otherwise, the headers may cause misinterpretation of returned data
scope.write(':MEASure:DELTAtime:DEF RISing,1,MIDD,RISing,1,MIDDle')
scope.write('CDIS')
scope.write(':MEASure:DELTAtime CHANnel{0},CHANnel{0}'.format(ScopeChannel1,ScopeChannel2))
time.sleep(1)
scope.write(':MEASure:RESults?')
resualt = scope.read()

tmin = float(resualt.split(',')[3])
tmax = float(resualt.split(',')[4])
mean = float(resualt.split(',')[5])
deviation = float(resualt.split(',')[6])

In [ ]:
offset_in_points_start = 0
offset_in_points_end = 1024
offset_in_points_jump = 128
sample_clock = 2.5e9
threshold = 20e-12
# skew control by points
for offset_in_points in np.arange(offset_in_points_start, offset_in_points_end, offset_in_points_jump):
    inst.send_scpi_cmd('INST:CHAN {0};:INSTrument:CHANnel:OFFSet {1}'.format(ScopeChannel2, offset_in_points))
    scope.write('MEASURE:CLEAR;:TIM:RANG 100E-8')
    time.sleep(1)
    scope.write('MEAS:DEF DELT,RIS,1,MIDD,RIS,1,MIDD;:MEASURE:DELTATIME WMEM1,FUNC1')
    time.sleep(2)
    scope.write(':MEASure:STATistics ON')
    scope.write(':MEASure:SENDvalid ON')
    scope.write('CHAN{0}:SCAL 0.02;:CHAN{0}:INP DC50'.format(ScopeChannel1))
    scope.write('CHAN{0}:SCAL 0.02;:CHAN{0}:INP DC50'.format(ScopeChannel2))
    scope.write('FUNC1:AVER CHAN{0};:FUNC1:DISP ON'.format(ScopeChannel2))
    time.sleep(1)
    scope.write('CDIS')
    scope.write(':MEASure:DELTAtime CHANnel{0},CHANnel{1}'.format(ScopeChannel1,ScopeChannel2))
    time.sleep(1)
    scope.write(':MEASure:RESults?')
    resualt = scope.read()
    temp_tmin = float(resualt.split(',')[3])
    temp_tmax = float(resualt.split(',')[4])
    temp_mean = float(resualt.split(',')[5])
    temp_deviation = float(resualt.split(',')[6])
    diff = abs(temp_mean - mean)
    offset_in_time = offset_in_points * (1 / (sample_clock))
    if (abs(diff - offset_in_time) > threshold):
        test_success = False
        print('Fail in offset {}, offset time {}, diff {}'.format(offset_in_points, offset_in_time, diff))
        # break
    else:
        print('Pass in offset {}, offset time {}, diff {}'.format(offset_in_points, offset_in_time, diff))

if (test_success):
    print('test successed')
else:
    print('test failed')

# Channels on the differenct DAC (Connect the channels to be compared to the scope channel 1 and channel 3)

In [ ]:
ScopeChannel1,ScopeChannel2 =1,3
inst.send_scpi_cmd(':INST:CHAN {0};:FREQ:RAST 2.5e9;:INIT:CONT ON;:SOUR:FUNC:MODE:SEGM 1;:OUTP ON'.format(ScopeChannel1))
inst.send_scpi_cmd(':INST:CHAN {0};:INIT:CONT ON;:SOUR:FUNC:MODE:SEGM 1;:OUTP ON'.format(ScopeChannel1))

test_success = True
scope.write('*RST;:CHAN{0}:DISP ON;:TIM:RANG 5E-9;:CHAN{1}:DISP ON;:CHAN{1}:SCAL 0.2;:CHAN{1}:INP DC50'.format(ScopeChannel2,ScopeChannel1))
scope.write(':MEASure:STATistics ON')
scope.write(':MEASure:SENDvalid ON')
scope.write('CHAN{0}:SCAL 0.2;:CHAN{0}:INP DC50'.format(ScopeChannel2))
scope.write('FUNC1:AVER CHAN2;:FUNC1:DISP ON')
time.sleep(1)
scope.write(
    ':SYSTem:HEADer OFF')  # When receiving numeric data into numeric variables, turn off the headers. Otherwise, the headers may cause misinterpretation of returned data
scope.write(':MEASure:DELTAtime:DEF RISing,1,MIDD,RISing,1,MIDDle')
scope.write('CDIS')
scope.write(':MEASure:DELTAtime CHANnel{0},CHANnel{0}'.format(ScopeChannel1,ScopeChannel2))
time.sleep(1)
scope.write(':MEASure:RESults?')
resualt = scope.read()

tmin = float(resualt.split(',')[3])
tmax = float(resualt.split(',')[4])
mean = float(resualt.split(',')[5])
deviation = float(resualt.split(',')[6])
offset_in_points_start = 0
offset_in_points_end = 1024
offset_in_points_jump = 128
sample_clock = 2.5e9
threshold = 20e-12
# skew control by points
for offset_in_points in np.arange(offset_in_points_start, offset_in_points_end, offset_in_points_jump):
    inst.send_scpi_cmd('INST:CHAN {};:INSTrument:CHANnel:OFFSet {}'.format(ScopeChannel2, offset_in_points))
    scope.write('MEASURE:CLEAR;:TIM:RANG 100E-8')
    time.sleep(1)
    scope.write('MEAS:DEF DELT,RIS,1,MIDD,RIS,1,MIDD;:MEASURE:DELTATIME WMEM1,FUNC1')
    time.sleep(2)
    scope.write(':MEASure:STATistics ON')
    scope.write(':MEASure:SENDvalid ON')
    scope.write('CHAN{0}:SCAL 0.02;:CHAN{0}:INP DC50'.format(ScopeChannel1))
    scope.write('CHAN{0}:SCAL 0.02;:CHAN{0}:INP DC50'.format(ScopeChannel2))
    scope.write('FUNC1:AVER CHAN{0};:FUNC1:DISP ON'.format(ScopeChannel2))
    time.sleep(1)
    scope.write('CDIS')
    scope.write(':MEASure:DELTAtime CHANnel{0},CHANnel{1}'.format(ScopeChannel1,ScopeChannel2))
    time.sleep(1)
    scope.write(':MEASure:RESults?')
    resualt = scope.read()
    temp_tmin = float(resualt.split(',')[3])
    temp_tmax = float(resualt.split(',')[4])
    temp_mean = float(resualt.split(',')[5])
    temp_deviation = float(resualt.split(',')[6])
    diff = abs(temp_mean - mean)
    offset_in_time = offset_in_points * (1 / (sample_clock))
    if (abs(diff - offset_in_time) > threshold):
        test_success = False
        print('Fail in offset {}, offset time {}, diff {}'.format(offset_in_points, offset_in_time, diff))
        # break
    else:
        print('Pass in offset {}, offset time {}, diff {}'.format(offset_in_points, offset_in_time, diff))

if (test_success):
    print('test successed')
else:
    print('test failed')

# Channels on the differenct DAC (Connect the channels to be compared to the scope channel 1 and channel 4)

In [ ]:
ScopeChannel1,ScopeChannel2 =1,4
inst.send_scpi_cmd(':INST:CHAN {0};:FREQ:RAST 2.5e9;:INIT:CONT ON;:SOUR:FUNC:MODE:SEGM 1;:OUTP ON'.format(ScopeChannel1))
inst.send_scpi_cmd(':INST:CHAN {0};:INIT:CONT ON;:SOUR:FUNC:MODE:SEGM 1;:OUTP ON'.format(ScopeChannel1))

test_success = True
scope.write('*RST;:CHAN{0}:DISP ON;:TIM:RANG 5E-9;:CHAN{1}:SCAL 0.2;:CHAN{1}:DISP ON;:CHAN{1}:INP DC50'.format(ScopeChannel2,ScopeChannel1))
scope.write(':MEASure:STATistics ON')
scope.write(':MEASure:SENDvalid ON')
scope.write('CHAN{0}:SCAL 0.2;:CHAN{0}:INP DC50'.format(ScopeChannel2))
scope.write('FUNC1:AVER CHAN2;:FUNC1:DISP ON')
time.sleep(1)
scope.write(
    ':SYSTem:HEADer OFF')  # When receiving numeric data into numeric variables, turn off the headers. Otherwise, the headers may cause misinterpretation of returned data
scope.write(':MEASure:DELTAtime:DEF RISing,1,MIDD,RISing,1,MIDDle')
scope.write('CDIS')
scope.write(':MEASure:DELTAtime CHANnel{0},CHANnel{0}'.format(ScopeChannel1,ScopeChannel2))
time.sleep(1)
scope.write(':MEASure:RESults?')
resualt = scope.read()

tmin = float(resualt.split(',')[3])
tmax = float(resualt.split(',')[4])
mean = float(resualt.split(',')[5])
deviation = float(resualt.split(',')[6])
offset_in_points_start = 0
offset_in_points_end = 1024
offset_in_points_jump = 128
sample_clock = 2.5e9
threshold = 20e-12
# skew control by points
for offset_in_points in np.arange(offset_in_points_start, offset_in_points_end, offset_in_points_jump):
    inst.send_scpi_cmd('INST:CHAN {};:INSTrument:CHANnel:OFFSet {}'.format(ScopeChannel2, offset_in_points))
    scope.write('MEASURE:CLEAR;:TIM:RANG 100E-8')
    time.sleep(1)
    scope.write('MEAS:DEF DELT,RIS,1,MIDD,RIS,1,MIDD;:MEASURE:DELTATIME WMEM1,FUNC1')
    time.sleep(2)
    scope.write(':MEASure:STATistics ON')
    scope.write(':MEASure:SENDvalid ON')
    scope.write('CHAN{0}:SCAL 0.02;:CHAN{0}:INP DC50'.format(ScopeChannel1))
    scope.write('CHAN{0}:SCAL 0.02;:CHAN{0}:INP DC50'.format(ScopeChannel2))
    scope.write('FUNC1:AVER CHAN{0};:FUNC1:DISP ON'.format(ScopeChannel2))
    time.sleep(1)
    scope.write('CDIS')
    scope.write(':MEASure:DELTAtime CHANnel{0},CHANnel{1}'.format(ScopeChannel1,ScopeChannel2))
    time.sleep(1)
    scope.write(':MEASure:RESults?')
    resualt = scope.read()
    temp_tmin = float(resualt.split(',')[3])
    temp_tmax = float(resualt.split(',')[4])
    temp_mean = float(resualt.split(',')[5])
    temp_deviation = float(resualt.split(',')[6])
    diff = abs(temp_mean - mean)
    offset_in_time = offset_in_points * (1 / (sample_clock))
    if (abs(diff - offset_in_time) > threshold):
        test_success = False
        print('Fail in offset {}, offset time {}, diff {}'.format(offset_in_points, offset_in_time, diff))
        # break
    else:
        print('Pass in offset {}, offset time {}, diff {}'.format(offset_in_points, offset_in_time, diff))

if (test_success):
    print('test successed')
else:
    print('test failed')

In [ ]:
ScopeChannel1 = 2
ScopeChannel2 =1
inst.send_scpi_cmd(':INST:CHAN {0};:FREQ:RAST 2.5e9;:INIT:CONT ON;:SOUR:FUNC:MODE:SEGM 1;:OUTP ON'.format(ScopeChannel1))
inst.send_scpi_cmd(':INST:CHAN {0};:INIT:CONT ON;:SOUR:FUNC:MODE:SEGM 1;:OUTP ON'.format(ScopeChannel1))

inst.send_scpi_cmd(':INST:CHAN {0};:FREQ:RAST 2.5e9;:INIT:CONT ON;:SOUR:FUNC:MODE:SEGM 1;:OUTP ON'.format(ScopeChannel2))
inst.send_scpi_cmd(':INST:CHAN {0};:INIT:CONT ON;:SOUR:FUNC:MODE:SEGM 1;:OUTP ON'.format(ScopeChannel2))
test_success = True
scope.write('*RST;:CHAN{0}:DISP ON;:TIM:RANG 5E-9;:CHAN{1}:DISP ON;:CHAN{1}:SCAL 0.2;:CHAN{1}:INP DC50'.format(ScopeChannel2,ScopeChannel1))
scope.write(':MEASure:STATistics ON')
scope.write(':MEASure:SENDvalid ON')
scope.write('CHAN{0}:SCAL 0.02;:CHAN{0}:INP DC50'.format(ScopeChannel1))
scope.write('CHAN{0}:SCAL 0.02;:CHAN{0}:INP DC50'.format(ScopeChannel2))
scope.write('FUNC1:AVER CHAN{0};:FUNC1:DISP ON'.format(ScopeChannel2))
scope.write('FUNC1:AVER CHAN{0};:FUNC1:DISP ON'.format(ScopeChannel1))
time.sleep(1)
scope.write(
    ':SYSTem:HEADer OFF')  # When receiving numeric data into numeric variables, turn off the headers. Otherwise, the headers may cause misinterpretation of returned data
scope.write(':MEASure:DELTAtime:DEF RISing,1,MIDD,RISing,1,MIDDle')
scope.write('CDIS')
scope.write(':MEASure:DELTAtime CHANnel{0},CHANnel{0}'.format(ScopeChannel1,ScopeChannel2))
time.sleep(1)
scope.write(':MEASure:RESults?')
resualt = scope.read()

tmin = float(resualt.split(',')[3])
tmax = float(resualt.split(',')[4])
mean = float(resualt.split(',')[5])
deviation = float(resualt.split(',')[6])
offset_in_points_start = 0
offset_in_points_end = 1024
offset_in_points_jump = 128
sample_clock = 2.5e9
threshold = 20e-12
# skew control by points
for offset_in_points in np.arange(offset_in_points_start, offset_in_points_end, offset_in_points_jump):
    inst.send_scpi_cmd('INST:CHAN {};:INSTrument:CHANnel:OFFSet {}'.format(ScopeChannel1, 0))
    inst.send_scpi_cmd('INST:CHAN {};:INSTrument:CHANnel:OFFSet {}'.format(ScopeChannel2, offset_in_points))
    scope.write('MEASURE:CLEAR;:TIM:RANG 100E-8')
    time.sleep(1)
    scope.write('MEAS:DEF DELT,RIS,1,MIDD,RIS,1,MIDD;:MEASURE:DELTATIME WMEM1,FUNC1')
    time.sleep(2)
    scope.write(':MEASure:STATistics ON')
    scope.write(':MEASure:SENDvalid ON')
    

    time.sleep(1)
    scope.write('CDIS')
    scope.write(':MEASure:DELTAtime CHANnel{0},CHANnel{1}'.format(ScopeChannel1,ScopeChannel2))
    time.sleep(1)
    scope.write(':MEASure:RESults?')
    resualt = scope.read()
    temp_tmin = float(resualt.split(',')[3])
    temp_tmax = float(resualt.split(',')[4])
    temp_mean = float(resualt.split(',')[5])
    temp_deviation = float(resualt.split(',')[6])
    diff = abs(temp_mean - mean)
    offset_in_time = offset_in_points * (1 / (sample_clock))
    if (abs(diff - offset_in_time) > threshold):
        test_success = False
        print('Fail in offset {}, offset time {}, diff {}'.format(offset_in_points, offset_in_time, diff))
        # break
    else:
        print('Pass in offset {}, offset time {}, diff {}'.format(offset_in_points, offset_in_time, diff))

if (test_success):
    print('test successed')
else:
    print('test failed')

# Disconnect

In [ ]:
disconnect()
# Close the connection to the instrument
scope.close()
resourceManager.close()
print('Disconnected')